In [20]:
import numpy as np 
import pandas as pd

In [21]:
books = pd.read_csv('BX-Books.csv', sep=";", on_bad_lines='skip', encoding='latin-1')
users = pd.read_csv('BX-Users.csv', sep=";", on_bad_lines='skip', encoding='latin-1')
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=";", on_bad_lines='skip', encoding='latin-1')

C:\Users\vaibhav\AppData\Local\Temp\ipykernel_2716\95832991.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('BX-Books.csv', sep=";", on_bad_lines='skip', encoding='latin-1')


In [22]:
books['Image-URL-M'][1]

'http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg'

In [23]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [24]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [25]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [26]:
books.rename(columns={"Book-Title":'Title',
                      'Book-Author':'Author',
                     "Year-Of-Publication":'Year',
                     "Image-URL-L":"URL"},inplace=True)

ratings.rename(columns={"Book-Rating": "Ratings"}, inplace=True)

In [27]:
books = books[['ISBN','Title', 'Author', 'Year', 'Publisher','URL']]

In [28]:
ratings['User-ID'].value_counts()

User-ID
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [29]:
x = ratings['User-ID'].value_counts() > 200
x[x].shape

(899,)

In [30]:
y= x[x].index
ratings = ratings[ratings['User-ID'].isin(y)]
ratings.shape

(526356, 3)

In [31]:
ratings_with_books = ratings.merge(books, on='ISBN')
print(ratings_with_books.head())

   User-ID        ISBN  Ratings  \
0   277427  002542730X       10   
1     3363  002542730X        0   
2    11676  002542730X        6   
3    12538  002542730X       10   
4    13552  002542730X        0   

                                               Title             Author  Year  \
0  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
1  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
2  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
3  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
4  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   

                   Publisher  \
0  John Wiley &amp; Sons Inc   
1  John Wiley &amp; Sons Inc   
2  John Wiley &amp; Sons Inc   
3  John Wiley &amp; Sons Inc   
4  John Wiley &amp; Sons Inc   

                                                 URL  
0  http://images.amazon.com/images/P/002542730X.0...  

In [32]:
ratings_with_books.shape

(487671, 8)

In [33]:
number_rating = ratings_with_books.groupby('Title')['Ratings'].count().reset_index()
print(number_rating.head())

                                               Title  Ratings
0   A Light in the Storm: The Civil War Diary of ...        2
1                              Always Have Popsicles        1
2               Apple Magic (The Collector's series)        1
3   Beyond IBM: Leadership Marketing and Finance ...        1
4   Clifford Visita El Hospital (Clifford El Gran...        1


In [34]:
number_rating.rename(columns={'Ratings':'No. of Ratings'},inplace=True)
print(number_rating.head())

                                               Title  No. of Ratings
0   A Light in the Storm: The Civil War Diary of ...               2
1                              Always Have Popsicles               1
2               Apple Magic (The Collector's series)               1
3   Beyond IBM: Leadership Marketing and Finance ...               1
4   Clifford Visita El Hospital (Clifford El Gran...               1


In [35]:
final_rating = ratings_with_books.merge(number_rating, on='Title')
print(final_rating.head())

   User-ID        ISBN  Ratings  \
0   277427  002542730X       10   
1     3363  002542730X        0   
2    11676  002542730X        6   
3    12538  002542730X       10   
4    13552  002542730X        0   

                                               Title             Author  Year  \
0  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
1  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
2  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
3  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
4  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   

                   Publisher  \
0  John Wiley &amp; Sons Inc   
1  John Wiley &amp; Sons Inc   
2  John Wiley &amp; Sons Inc   
3  John Wiley &amp; Sons Inc   
4  John Wiley &amp; Sons Inc   

                                                 URL  No. of Ratings  
0  http://images.amazon.com/images/P/0

In [36]:
final_rating.shape

(487671, 9)

In [51]:
final_rating = final_rating[final_rating['No. of Ratings'] >= 50]
final_rating.drop_duplicates(['User-ID','Title'],inplace=True)
final_rating.shape

C:\Users\vaibhav\AppData\Local\Temp\ipykernel_2716\215743997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rating.drop_duplicates(['User-ID','Title'],inplace=True)


(59850, 9)

In [53]:
book_pivot = final_rating.pivot_table(columns='User-ID', index='Title', values="Ratings")

In [54]:
book_pivot.fillna(0, inplace=True)
print(book_pivot)

User-ID                                             254     2276    2766    \
Title                                                                        
1984                                                   9.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0    10.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
84 Charing Cross Road                                  0.0     0.0     0.0   
...                                                    ...     ...     ...   
Year of Wonders                                        0.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
Zoya                                                   0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0

In [55]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

book_names = book_pivot.index
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [56]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6 )
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [57]:
suggestion

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

In [58]:
book_pivot.iloc[241,:]

User-ID
254       9.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
275970    9.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: Harry Potter and the Sorcerer's Stone (Book 1), Length: 888, dtype: float64

In [59]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='Title')


In [61]:
import pickle
pickle.dump(model,open('Model.pkl','wb'))
pickle.dump(book_names,open('Book_Names.pkl','wb'))
pickle.dump(final_rating,open('Final_Rating.pkl','wb'))
pickle.dump(book_pivot,open('Book_Pivot.pkl','wb'))